In [3]:
# Imports
from aalpy.SULs import MealySUL
from aalpy.oracles import StatePrefixEqOracle

from Applications import label_sequences_with_correct_model
from DataProcessing import get_coffee_machine, generate_data_from_automaton, split_train_validation, tokenize
from RNNClassifier import RNNClassifier
from TrainAndExtract import extract_finite_state_transducer

In [6]:
# Get the coffee machine FSM used for data generation
ground_truth_model = get_coffee_machine()

# Get input and output alphabet 
input_al = ground_truth_model.get_input_alphabet()
output_al = {output for state in ground_truth_model.states for output in state.output_fun.values()}

# Create the small initial training set
train_seq, train_labels = generate_data_from_automaton(ground_truth_model, input_al,
                                                       num_examples=1000, lens=(3, 6, 9))

In [7]:
# Define a number of RNNs that will be used for learning-based testing
# Each NN will be trained with the training data set.
# Models will be mined from all RNNs and cross checked for conformance.
# Cases of non-conformance are added to the training set.
NUM_RNNs = 4

In [8]:
# While the input-output behaviour of all trained neural networks is different
iteration = 0
while True:
    iteration += 1
    print(f'Learning/extraction round: {iteration}')

    trained_networks = []

    x_train, y_train, x_test, y_test = split_train_validation(train_seq, train_labels, 0.8, uniform=True)

    # Train all neural networks with same parameters
    for i in range(NUM_RNNs):
        rnn = RNNClassifier(input_al, output_dim=len(output_al), num_layers=2, hidden_dim=40,
                            x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test,
                            batch_size=32, nn_type='GRU')
        print(f'Starting training of RNN {i}')
        rnn.train(epochs=150, stop_acc=1.0, stop_epochs=3, verbose=False)
        trained_networks.append(rnn)

    learned_automatons = []

    # Extract automaton for each neural network
    for i, rnn in enumerate(trained_networks):
        print(f'Starting extraction of the automaton from RNN {i}')
        learned_automaton = extract_finite_state_transducer(rnn, input_al, output_al, max_learning_rounds=6,
                                                            print_level=0)
        learned_automatons.append(learned_automaton)

    learned_automatons.sort(key=lambda x: len(x.states), reverse=True)

    # Select one automaton as a basis for conformance-checking. You can also do conformance checking with all pairs
    # of learned automata.

    base_sul = MealySUL(learned_automatons[0])

    # Select the eq. oracle

    eq_oracle = StatePrefixEqOracle(input_al, base_sul, walks_per_state=100, walk_len=50)

    cex_set = set()

    # Try to find cases of non-conformance between learned automatons.
    for la in learned_automatons[1:]:
        for i in range(200):
            cex = eq_oracle.find_cex(la)
            if cex:
                cex_set.add(tuple(cex))

    # If there were no counterexamples between any learned automata, we end the procedure
    if not cex_set:
        for i, la in enumerate(learned_automatons):
            print(f'Size of automata {i}: {len(la.states)}')
        print(learned_automatons[-1])
        print('No counterexamples between extracted automata found.')
        break

    # Ask ground truth model for correct labels
    new_x, new_y = label_sequences_with_correct_model(ground_truth_model, cex_set)

    print(f'Adding {len(cex_set)} new examples to training data.')
    new_x = tokenize(new_x, input_al)
    new_y = tokenize(new_y, output_al)

    train_seq.extend(new_x)
    train_labels.extend(new_y)
    print(f'Size of training data: {len(train_seq)}')

Learning/extraction round: 1
Starting training of RNN 0
Starting training of RNN 1
Starting training of RNN 2
Starting training of RNN 3
Starting extraction of the automaton from RNN 0
Starting extraction of the automaton from RNN 1
Starting extraction of the automaton from RNN 2
Starting extraction of the automaton from RNN 3
Adding 260 new examples to training data.
Size of training data: 1262
Learning/extraction round: 2
Starting training of RNN 0
Starting training of RNN 1
Starting training of RNN 2
Starting training of RNN 3
Starting extraction of the automaton from RNN 0
Starting extraction of the automaton from RNN 1
Starting extraction of the automaton from RNN 2
Starting extraction of the automaton from RNN 3
Adding 308 new examples to training data.
Size of training data: 1570
Learning/extraction round: 3
Starting training of RNN 0
Starting training of RNN 1
Starting training of RNN 2
Starting training of RNN 3
Starting extraction of the automaton from RNN 0
Starting extracti